# HE Slide Analysis: Muscle Cell and Nuclei Segmentation

This notebook performs two main tasks on HE stained slide images:
1.  **Muscle Cell Segmentation:** Identifies individual muscle cells (pink/red cytoplasm) using Cellpose.
2.  **Nuclei Segmentation:** Identifies brown nuclei within the cells using Color Deconvolution followed by Cellpose.

## 1. Setup and Imports

In [2]:
pwd

'/n/projects/zyu/Jupyter/xinjian'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tifffile
from skimage import io, color, restoration
from cellpose import models
from cellpose import plot
import napari
import os
import torch

# Check for GPU
use_gpu = torch.cuda.is_available()
print(f"GPU Available: {use_gpu}")

In [1]:
from dask_jobqueue import SLURMCluster
from dask.distributed import Client
import time

cluster = SLURMCluster(
    cores=1,                      # Number of cores per job
    memory="64GB",                # Memory per job
    queue="gpu",         # Queue/partition name
    walltime="2:00:00",           # Job time limit
    job_extra_directives =[
        '--gres=gpu:rtx4500:1',  # Number of GPUs per job
        '--qos=rtx4500', # Quality of Service for GPU jobs
    ],
)

client = Client(cluster)
print(client)

cluster.scale(jobs=1)  # Request 1 job
print(client.cluster)

<Client: 'tcp://10.0.53.5:45757' processes=0 threads=0, memory=0 B>
SLURMCluster(21bc54ca, 'tcp://10.0.53.5:45757', workers=0, threads=0, memory=0 B)


2026-01-27 13:17:55,607 - tornado.application - ERROR - Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <tornado.platform.asyncio.AsyncIOMainLoop object at 0x7fd4e3754260>>, <Task finished name='Task-49' coro=<SpecCluster._correct_state_internal() done, defined at /n/projects/zyu/Jupyter/.pixi/envs/default/lib/python3.12/site-packages/distributed/deploy/spec.py:352> exception=RuntimeError('Command exited with non-zero exit code.\nExit code: 1\nCommand:\nsbatch /tmp/tmpzrq2m2kd.sh\nstdout:\n\nstderr:\nsbatch: error: Batch job submission failed: Invalid qos specification\n\n')>)
Traceback (most recent call last):
  File "/n/projects/zyu/Jupyter/.pixi/envs/default/lib/python3.12/site-packages/tornado/ioloop.py", line 758, in _run_callback
    ret = callback()
          ^^^^^^^^^^
  File "/n/projects/zyu/Jupyter/.pixi/envs/default/lib/python3.12/site-packages/tornado/ioloop.py", line 782, in _discard_future_result
    future.result()
  File "/n/projec

In [ ]:
# Load Image (Supports JPG, PNG, TIF)
IMAGE_PATH = r"u:\zyu\Jupyter\xinjian\your_image.tif" # Update to your TIF file path

# Use tifffile for TIF or skimage.io for other formats
if IMAGE_PATH.lower().endswith(('.tif', '.tiff')):
    img_rgb = tifffile.imread(IMAGE_PATH)
else:
    img_rgb = io.imread(IMAGE_PATH)
    
print(f"Loaded: {IMAGE_PATH} | Shape: {img_rgb.shape}")

# Simple corrections: Channels first -> Channels last, Remove Alpha
if img_rgb.ndim == 3 and img_rgb.shape[0] < 5: 
    img_rgb = np.transpose(img_rgb, (1, 2, 0))
if img_rgb.shape[-1] == 4: 
    img_rgb = img_rgb[..., :3]

# --- CHANNEL CHECK ---
# Verify if channels are RGB or BGR
fig, ax = plt.subplots(1, 4, figsize=(20, 5))
ax[0].imshow(img_rgb); ax[0].set_title("Composite")
ax[1].imshow(img_rgb[..., 0], cmap='gray'); ax[1].set_title("Channel 0 (Should be Red/Eosin)")
ax[2].imshow(img_rgb[..., 1], cmap='gray'); ax[2].set_title("Channel 1 (Green)")
ax[3].imshow(img_rgb[..., 2], cmap='gray'); ax[3].set_title("Channel 2 (Should be Blue/Hem/Nuclei)")
for a in ax: a.axis('off')
plt.show()

## Part 1: Muscle Cell Segmentation (Cellpose)

We use Cellpose to segment the muscle cells. 
*   **Model:** `cyto2` (works well for cytoplasm and general cells).
*   **Target:** Red/Pink areas.

In [ ]:
# 1. Muscle Segmentation
# Set expected diameter (in pixels) to speed up processing
# Set to None for auto-estimation (slower)
CELL_DIAMETER = 80 # <-- CHANGE THIS to your cell size

model_cyto = models.Cellpose(gpu=use_gpu, model_type='cyto2')

print(f"Segmenting muscle cells using diameter={CELL_DIAMETER}...")
masks_cells, flows, styles, diams = model_cyto.eval(
    img_rgb, 
    diameter=CELL_DIAMETER, 
    flow_threshold=0.4, 
    channels=[1, 3]
)

# Quick verify
plt.figure(figsize=(6,6))
plot.show_segmentation(plt.gca(), img_rgb, masks_cells, flows[0], channels=[1,3])
plt.title(f"Muscle Cells: {masks_cells.max()}")
plt.show()

# View with napari
print("\nOpen Napari viewer for Muscle Cell Segmentation...")
viewer_cells = napari.Viewer()
viewer_cells.add_image(img_rgb, name='RGB Image')
viewer_cells.add_labels(masks_cells, name='Muscle Cells')
print(f"Found {masks_cells.max()} muscle cells")

## Part 2: Nuclei Segmentation (Color Deconvolution + Cellpose)

Since the nuclei are described as "brown" (likely DAB stained or dense Hematoxylin), simple RGB thresholding might fail. We will use **Color Deconvolution** to separate the stains.
*   **HED Separation:** Hematoxylin (Blue), Eosin (Pink), DAB (Brown).